In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as dataset
import torchvision.transforms as transforms
import numpy as np

In [59]:
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 96, 11, 4, 2),  # 11+4*54<228=224+2*2
            nn.ReLU(),
            nn.MaxPool2d(3, 2),  # 3+2*26=55
            nn.Conv2d(96, 192, 5, 1, 2),  # 5+26=31=27+2*2
            nn.ReLU(),
            nn.MaxPool2d(3, 2),  # 3+2*12=27
            nn.Conv2d(192, 384, 3, 1, 1),  #3+12=15=13+1*2
            nn.ReLU(),
            nn.Conv2d(384, 256, 3, 1, 1),  #3+12=15=13+1*2
            nn.ReLU(),
            nn.Conv2d(256, 128, 3, 1, 1),  #3+12=15=13+1*2
#             nn.Conv2d(256, 256, 3, 1, 1),  #3+12=15=13+1*2
            nn.ReLU(),
            nn.MaxPool2d(3, 2),  # 3+2*5=13
#             nn.MaxPool2d(5, 2, 1),  # 5+2*5=13+1*2
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(128 * 6 * 6, 4096),
#             nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(),
            nn.Dropout(),
            # nn.Linear(4096, 4096),  # 1024
            nn.Linear(4096, 1024),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(1024, 10))
        # nn.Linear(4096, 10))

    def forward(self, x):
        x = self.features(x)
#         x = self.classifier(x.view(-1, 256 * 6 * 6))
        x = self.classifier(x.view(-1, 128 * 6 * 6))
        return x

In [3]:
transform = transforms.Compose([
    transforms.Resize(32),
    transforms.RandomHorizontalFlip(),
    transforms.CenterCrop(32), # 从中心裁剪
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.465, 0.406], [0.229, 0.224, 0.225]),
])

In [16]:
train_data = dataset.SVHN(root='data',
                          split='train',
                          transform=transform,
                          download=False)
test_data = dataset.SVHN(root='data',
                         split='test',
                         transform=transform,
                         download=False)

In [12]:
# s=0
# for i in range(0,11):
#     s=np.sum(train_data.labels==i)
#     print(s)
# print(train_data.data.shape)
# s=0
# for i in range(0,11):
#     s=np.sum(test_data.labels==i)
#     print(s)
# print(test_data.data.shape)


print(train_data[0][1])
print(train_data[0][0].shape)
print(train_data[0][0][0].shape)
print(train_data[0][0][2].shape)

print(train_data.data[49999].shape)

print(type(train_data[0]))

# train_data=[train_data[i][0] for i in range(len(train_data))]
# test_data=[test_data[i][0] for i in range(len(test_data))]

1
torch.Size([3, 32, 32])
torch.Size([32, 32])
torch.Size([32, 32])
(3, 32, 32)
<class 'tuple'>


In [33]:
print(type(train_data.data))
print(type(train_data.labels))
print(train_data.data.shape)
d=train_data.data.reshape(73257,-1)
print(train_data.labels.shape)
print(train_data.data[23])
print(d[23])

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(73257, 3, 32, 32)
(73257,)
[[[115 115 112 ... 122 123 124]
  [106 102  97 ... 122 121 121]
  [102  98  88 ... 121 121 120]
  ...
  [102  99  93 ... 123 123 125]
  [106 104 101 ... 123 123 125]
  [111 111 111 ... 124 124 125]]

 [[112 113 108 ... 120 121 122]
  [103  98  93 ... 120 119 119]
  [ 98  92  82 ... 121 119 118]
  ...
  [100  95  89 ... 122 121 120]
  [105 101  97 ... 122 121 120]
  [110 108 107 ... 122 122 120]]

 [[105 105 102 ... 123 124 125]
  [ 96  92  87 ... 123 123 122]
  [ 91  86  76 ... 122 121 121]
  ...
  [ 98  94  88 ... 124 125 126]
  [103  99  96 ... 126 126 126]
  [108 106 106 ... 126 127 126]]]
[115 115 112 ... 126 127 126]


In [81]:
# train_data=[train_data[i] for i in range(320)]
# test_data=[test_data[i] for i in range(32)]
train_loader = torch.utils.data.DataLoader(train_data,
                                           batch_size=16,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data,
                                          batch_size=16,
                                          shuffle=True)
i=0
for batch in train_loader:
    images, labels = batch
#     print(images)
#     print(images.shape)
#     print(labels)
    i+=1
#     break
print(i)

tensor([[[[-0.0458, -0.0458, -0.0458,  ..., -1.2103, -1.2103, -1.2103],
          [-0.0458, -0.0458, -0.0458,  ..., -1.2103, -1.2103, -1.2103],
          [-0.0458, -0.0458, -0.0458,  ..., -1.2103, -1.2103, -1.2103],
          ...,
          [-0.0801, -0.0801, -0.0801,  ..., -1.3130, -1.3130, -1.3130],
          [-0.0801, -0.0801, -0.0801,  ..., -1.3130, -1.3130, -1.3130],
          [-0.0801, -0.0801, -0.0801,  ..., -1.3130, -1.3130, -1.3130]],

         [[ 0.0600,  0.0600,  0.0600,  ..., -1.1305, -1.1305, -1.1305],
          [ 0.0600,  0.0600,  0.0600,  ..., -1.1305, -1.1305, -1.1305],
          [ 0.0600,  0.0600,  0.0600,  ..., -1.1305, -1.1305, -1.1305],
          ...,
          [ 0.0425,  0.0425,  0.0425,  ..., -1.1655, -1.1655, -1.1655],
          [ 0.0425,  0.0425,  0.0425,  ..., -1.1655, -1.1655, -1.1655],
          [ 0.0425,  0.0425,  0.0425,  ..., -1.1655, -1.1655, -1.1655]],

         [[ 0.2348,  0.2348,  0.2348,  ..., -0.8284, -0.8284, -0.8284],
          [ 0.2348,  0.2348,  

In [80]:
train_len = len(train_data)
test_len = len(test_data)
print("AlexNet")
print(train_len, test_len)

if __name__ == '__main__':

    model = AlexNet()
    # model = model.cuda()
    optimzer = optim.SGD(model.parameters(), lr=0.01)

    print("batch_b16 ")
    for epoch in range(10):
        total_loss = 0
        train_correct = 0
        test_correct = 0
        cnt = 0
        for batch in train_loader:
#             cnt += len(data)
#             print("processing: %d/%d" % (cnt, len(train_loader.dataset)))
            images, labels = batch
            outs = model(images)
            loss = F.cross_entropy(outs, labels)
            # outs = model(images.cuda())
            # loss = F.cross_entropy(outs, labels.cuda())
            total_loss += loss.item()
            optimzer.zero_grad()
            loss.backward()
            optimzer.step()
            # train_correct += outs.argmax(dim=1).eq(labels.cuda()).sum().item()
            train_correct += outs.argmax(dim=1).eq(labels).sum().item()

        for batch in test_loader:
            images, labels = batch
            out = model(images)
            test_correct += out.argmax(dim=1).eq(labels).sum().item()
            # out = model(images.cuda())
            # test_correct = out.argmax(dim=1).eq(labels.cuda()).sum().item()
        train_correct = train_correct / train_len
        test_correct = test_correct / test_len
        print(epoch, ' -- total_loss:', total_loss, '  train_correct:',
              train_correct, '  test_correct:', test_correct)

    out_output = []
    out_target = []
    out_data = []
    # cnt=0
    # print("save final train output")
    for batch in train_loader:
        # cnt += len(data)
        # print("processing: %d/%d" % (cnt, len(train_loader.dataset)))
        images, labels = batch
        outs = model(images)

        output_np = outs.data.numpy()
        target_np = labels.data.numpy()
        data_np = images.data.numpy()
        out_output.append(output_np)
        out_target.append(target_np[:, np.newaxis])  #(10,)变(10,1)
        out_data.append(np.squeeze(data_np))

    output_array = np.concatenate(out_output, axis=0)
    target_array = np.concatenate(out_target, axis=0)
    data_array = np.concatenate(out_data, axis=0)
    # 不允许序列化
    np.save('out/AlexNet_b16_output.npy',
            output_array,
            allow_pickle=False)
    np.save('out/AlexNet_b16_target.npy',
            target_array,
            allow_pickle=False)
    # np.save('out/AlexNet_b8_data.npy', data_array, allow_pickle=False)

    out_output = []
    out_target = []
    out_data = []
    # cnt=0
    # print("save final test output")
    for batch in test_loader:
        # cnt += len(data)
        # print("processing: %d/%d" % (cnt, len(train_loader.dataset)))
        images, labels = batch
        outs = model(images)

        output_np = outs.data.numpy()
        target_np = labels.data.numpy()
        data_np = images.data.numpy()
        out_output.append(output_np)
        out_target.append(target_np[:, np.newaxis])  #(10,)变(10,1)
        out_data.append(np.squeeze(data_np))

    output_array = np.concatenate(out_output, axis=0)
    target_array = np.concatenate(out_target, axis=0)
    data_array = np.concatenate(out_data, axis=0)
    # 不允许序列化
    np.save('out/AlexNet_b16_test_output.npy',
            output_array,
            allow_pickle=False)
    np.save('out/AlexNet_b16_test_target.npy',
            target_array,
            allow_pickle=False)
    # np.save('out/AlexNet_b8_test_data.npy', data_array, allow_pickle=False)

AlexNet
320 32
batch_b16 
0  -- total_loss: 45.980289697647095   train_correct: 0.146875   test_correct: 0.125
1  -- total_loss: 45.87960410118103   train_correct: 0.1375   test_correct: 0.09375
2  -- total_loss: 45.790281772613525   train_correct: 0.15   test_correct: 0.1875
3  -- total_loss: 45.67021083831787   train_correct: 0.184375   test_correct: 0.25
4  -- total_loss: 45.579591512680054   train_correct: 0.190625   test_correct: 0.25
5  -- total_loss: 45.49575853347778   train_correct: 0.190625   test_correct: 0.25
6  -- total_loss: 45.413344860076904   train_correct: 0.19375   test_correct: 0.25
7  -- total_loss: 45.32223320007324   train_correct: 0.19375   test_correct: 0.25
8  -- total_loss: 45.24107336997986   train_correct: 0.19375   test_correct: 0.25
9  -- total_loss: 45.12474989891052   train_correct: 0.196875   test_correct: 0.25


In [ ]:
print()